In [2]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass 

In [6]:
x = jnp.array([
    [1,1],
    [2,2],
    [3,3],
    [4,4]
])
pts = jnp.stack((x, x + 0.1, x + 0.3))
pts, pts.shape

(Array([[[1. , 1. ],
         [2. , 2. ],
         [3. , 3. ],
         [4. , 4. ]],
 
        [[1.1, 1.1],
         [2.1, 2.1],
         [3.1, 3.1],
         [4.1, 4.1]],
 
        [[1.3, 1.3],
         [2.3, 2.3],
         [3.3, 3.3],
         [4.3, 4.3]]], dtype=float32, weak_type=True),
 (3, 4, 2))

In [14]:
# (2,4,2)
pts[0] - jnp.concat((pts[:0], pts[1:]))

Array([[[-0.10000002, -0.10000002],
        [-0.0999999 , -0.0999999 ],
        [-0.0999999 , -0.0999999 ],
        [-0.0999999 , -0.0999999 ]],

       [[-0.29999995, -0.29999995],
        [-0.29999995, -0.29999995],
        [-0.29999995, -0.29999995],
        [-0.3000002 , -0.3000002 ]]], dtype=float32, weak_type=True)

In [20]:
jnp.argmin(jnp.linalg.norm(pts[0] - jnp.concat((pts[:0], pts[1:])), axis=2), axis=0)

Array([0, 0, 0, 0], dtype=int32)

In [23]:
jnp.concat((pts[:0], pts[1:]))[:,jnp.array([0, 0, 0, 0])]

Array([[[1.1, 1.1],
        [1.1, 1.1],
        [1.1, 1.1],
        [1.1, 1.1]],

       [[1.3, 1.3],
        [1.3, 1.3],
        [1.3, 1.3],
        [1.3, 1.3]]], dtype=float32, weak_type=True)

In [24]:
pts[0]

Array([[1., 1.],
       [2., 2.],
       [3., 3.],
       [4., 4.]], dtype=float32, weak_type=True)

In [26]:
def find_closest_pt(pts, i):
    other_pts = jnp.concat((jax.lax.dynamic_slice(pts, (0, i)), jax.lax.dynamic_slice(pts, i+1)))
    dists = jnp.linalg.norm(pts[i] - other_pts, axis=2)
    closest_pts_indices = jnp.argmin(dists, axis=0)
    # closest_pts = dists[closest_pts_indices]
    closest_pts = jax.lax.dynamic_slice_in_dim(other_pts, closest_pts_indices) 
    
    return pts, closest_pts 
n_agents = pts.shape[0]
_, closest_pt = jax.lax.scan(find_closest_pt, pts, jnp.arange(n_agents))
closest_pt, closest_pt.shape

TypeError: object of type 'int' has no len()